<a href="https://colab.research.google.com/github/williserdman/python_playground/blob/main/basic_text_classification/main_tf_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

In [ ]:
print("System version:", sys.version)
import tensorflow as tf

In [ ]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [ ]:
print("Tensorflow version: ", tf.__version__)
print("Tensorflow eager mode: ", tf.executing_eagerly())
print("Tensorflow Hub version: ", hub.__version__)
print(
    "GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE"
)

In [ ]:
train_ds, val_ds, test_ds = tfds.load(
    name="imdb_reviews",
    split=("train[:60%]", "train[60%:]", "test"),
    as_supervised=True,
)

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_ds.batch(10)))
print(train_examples_batch)
print(train_labels_batch)

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
example_hub_layer = hub_layer(train_examples_batch[:3])
print(example_hub_layer)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(1))

In [ ]:
print(model.summary())
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)